In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import json
import timeit

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
X_train = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SPAM/X_train_fSP.pkl')
y_train = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SPAM/y_train_fSP.pkl')
X_test = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SPAM/X_test_fSP.pkl')
y_test = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/Full Text SPAM/y_test_fSP.pkl')

## TF-IDF with top 15 features
X_train_tfidf = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF SPAM/X_train_tfSP.pkl')
X_test_tfidf = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF SPAM/X_test_tfSP.pkl')

## TF-IDF GloVe with top 15 features
X_train_tfidf_glove = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF Glove SPAM/X_train_tfglSP.pkl')
X_test_tfidf_glove = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF Glove SPAM/X_test_tfglSP.pkl')

## TF-IDF FastText (cc) with top 15 features
X_train_tfidf_cc = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF CC SPAM/X_train_tfccSP.pkl')
X_test_tfidf_cc = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/TFIDF CC SPAM/X_test_tfccSP.pkl')

X_train_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SPAM/X_train_infSP.pkl')
X_test_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SPAM/X_test_infSP.pkl')
y_train_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SPAM/y_train_infSP.pkl')
y_test_infersent = pd.read_pickle('/content/gdrive/My Drive/BT4222/Codes/Cleaned Input Data/InferSent SPAM/y_test_infSP.pkl')

Load Models

In [ ]:
model_tfidf = load_model('/content/gdrive/My Drive/BT4222/Code (Final Submission)/Spam Models/Saved Models/model_Tfidf')
model_tfidf_glove  = load_model('/content/gdrive/My Drive/BT4222/Code (Final Submission)/Spam Models/Saved Models/model_Tfidf_glove')
model_tfidf_cc  = load_model('/content/gdrive/My Drive/BT4222/Code (Final Submission)/Spam Models/Saved Models/model_Tfidf_cc')
model_infersent = load_model('/content/gdrive/My Drive/BT4222/Code (Final Submission)/Spam Models/Saved Models/model_Infersent')

Predict on Test set

In [ ]:
datasets = {'Tfidf' : {'X_train' : X_train_tfidf,
                       'y_train' : y_train,
                       'X_test' : X_test_tfidf,
                       'y_test' : y_test,
                       'Model' : model_tfidf},
            'Tfidf_cc' : {'X_train' : X_train_tfidf_cc,
                          'y_train' : y_train,
                          'X_test' : X_test_tfidf_cc,
                          'y_test' : y_test,
                          'Model' : model_tfidf_cc},
            'Tfidf_glove' : {'X_train' : X_train_tfidf_glove, 
                             'y_train' : y_train, 
                             'X_test' : X_test_tfidf_glove,
                             'y_test' : y_test, 
                             'Model' : model_tfidf_glove},
            'Infersent' : {'X_train' : X_train_infersent,
                           'y_train' : y_train_infersent, 
                           'X_test' : X_test_infersent,
                           'y_test' : y_test_infersent, 
                           'Model' : model_infersent}
            }

results = {'Tfidf' : {'Accuracy' : 0, 'AUC' : 0, 'FPR' : 0, 'Sensitivity' : 0, 'F-Score' : 0 },
           'Tfidf_cc' : {'Accuracy' : 0, 'AUC' : 0, 'FPR' : 0, 'Sensitivity' : 0, 'F-Score' : 0},
           'Tfidf_glove' : {'Accuracy' : 0, 'AUC' : 0, 'FPR' : 0,'Sensitivity' : 0, 'F-Score' : 0},
           'Infersent' : {'Accuracy' : 0, 'AUC' : 0, 'FPR' : 0,'Sensitivity' : 0, 'F-Score' : 0}}

In [ ]:
def predict(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    y_pred = (model.predict(X_test)> 0.5).astype("int32")
    y_pred_prob = model.predict(X_test)    
    auc = roc_auc_score(y_test, y_pred_prob)
    #print("AUC {}".format(metrics.roc_auc_score(y_test, y_pred_prob)))    
    #print("Recall: {}".format(recall_score(y_test, y_pred_class, average='weighted')))
    #print("Precision: {}".format(precision_score(y_test, y_pred_class, average='weighted')))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fpr = fp/(fp+tn)
    tpr = tp/(tp + fn)
    #print('False Positive Rate: {}'.format(fp/(fp+tn)))
    f1 = f1_score(y_test, y_pred)
    return (accuracy, auc, fpr, tpr, f1)

for key, value in datasets.items():
    model = value['Model']
    X_test = value['X_test']
    y_test = value['y_test']
    scores =  predict(model, X_test, y_test)
    results[key]['Accuracy'] = scores[0]
    results[key]['AUC'] = scores[1]
    results[key]['FPR'] = scores[2]
    results[key]['Sensitivity'] = scores[3]
    results[key]['F-Score'] = scores[4]

270/270 [==============================] - 1s 2ms/step - loss: 0.6775 - accuracy: 0.6796


In [ ]:
print(json.dumps(results, indent=4, sort_keys=True))

{
    "Infersent": {
        "AUC": 0.7028037725681988,
        "Accuracy": 0.6795541644096375,
        "F-Score": 0.5296523517382413,
        "FPR": 0.1718358697746099,
        "Sensitivity": 0.45412039742840443
    },
    "Tfidf": {
        "AUC": 0.985682481532226,
        "Accuracy": 0.9451906681060791,
        "F-Score": 0.9383617193836172,
        "FPR": 0.04578038449006191,
        "Sensitivity": 0.9340060544904137
    },
    "Tfidf_cc": {
        "AUC": 0.9880196614759292,
        "Accuracy": 0.9505994915962219,
        "F-Score": 0.9441500203832042,
        "FPR": 0.036656891495601175,
        "Sensitivity": 0.9348133198789101
    },
    "Tfidf_glove": {
        "AUC": 0.9768149743915506,
        "Accuracy": 0.9285134673118591,
        "F-Score": 0.9207554711701809,
        "FPR": 0.07249918540241121,
        "Sensitivity": 0.9297679112008073
    }
}
